# Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statistics 
from sklearn.preprocessing import StandardScaler
print(tf.__version__)
import random

#load data

In [ ]:
dicti={
 'Andhra_Pradesh':[0,13],
 'Arunachal_Pradesh':[13,38],
 'Assam':[38,71],
 'Bihar':[71,109],
 'Chattisgarh':[109,137],
 'Goa':[137,139],
 'Gujarat':[139,172],
 'Haryana':[172,194],
 'Himanchal_Pradesh':[194,206],
 'Jharkhand':[206,230],
 'Karnataka':[230,260],
 'Kerala':[260,274],
 'Madhya_Pradesh':[274,329],
 'Maharashtra':[329,365],
 'Manipur':[365,381],
 'Meghalaya':[381,392],
 'Mizoram':[392,400],
 'Nagaland':[400,412],
 'Odisha':[412,442],
 'Punjab':[442,464],
 'Rajasthan':[464,497],
 'Sikkim':[497,501],
 'Tamil_Nadu':[501,539],
 'Telangana':[539,572],
 'Tripura':[572,580],
 'Uttar_Pradesh':[580,655],
 'Uttarakhand':[655,668],
 'West_Bengal':[668,691],
 'Andaman':[691,694],
 'Chandigarh':[694,695],
 'Daman':[695,698],
 'Jammu':[698,718],
 'Ladakh':[718,720],
 'Lakshadweep':[720,721],
 'Delhi':[721,732],
 'Puducherry':[732,736]
}

In [ ]:
!gdown --id 1pQzv9GfJEak6EodyEgNnLGJZpt0iweq9
data=np.load('/content/3ddistrict.npy')
data.shape
# !gdown --id 1NBymn3rsbOLt5DQcr0bX1Ln08MNrazLb
# datas=np.load('/content/3dstate.npy')
# datas.shape
print('number of nans: ',np.count_nonzero(np.isnan(data)))
for dis in range(736):
  for feat in range(5):
    for day in range(0,357):
      if np.isnan(data[feat,day,dis]):
        for dist in range(10):
          l=day-dist
          r=day+dist
          if ~np.isnan(data[feat,l,dis]): 
            data[feat,day,dis]=data[feat,l,dis]
            break
          elif ~np.isnan(data[feat,r,dis]): 
            data[feat,day,dis]=data[feat,r,dis]
            break

      

      
print('number of nans',np.count_nonzero(np.isnan(data)))
scaler = StandardScaler()
scaler.fit(np.expand_dims(data.flatten('F'),1))
data2=np.zeros((5,357,736))
for dist in range(736):
  temp=data[:,:,dist]
  # print(temp.shape)
  data2[:,:,dist]=scaler.transform(temp)
  # data2[:,4,dist]= data[:,4,dist]
  # data2[:,:,dist]=temp/1000
data2=data2.swapaxes(0,1)
print('shape of data: ',data2.shape)
data3=data2

Downloading...
From: https://drive.google.com/uc?id=1pQzv9GfJEak6EodyEgNnLGJZpt0iweq9
To: /content/3ddistrict.npy
10.5MB [00:00, 92.8MB/s]
number of nans:  69
number of nans 0
shape of data:  (357, 5, 736)


In [ ]:
data3=data3[:,:,329:365]

In [ ]:
data3.shape

(357, 5, 736)

#Data hyperparamets

In [ ]:
batch_size = 32
shuffle_buffer_size = 1000
traindays=200
num_of_districts=736
window_size=21  #same as n_past
n_past = 21
n_future = 7
n_features = 2

#Window data



In [ ]:
window_size=n_past
def windowed_dataset(series, window_size, batch_size, shuffle_buffer,test):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + n_future, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + n_future))
  if test : dataset = dataset.map(lambda window: (window[:-n_future], window[-n_future:]))
  else:
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-n_future], window[-n_future:]))
  return dataset
test=data3[traindays:,:,:]
data2=data3[:traindays,:,:]
X,Y=[],[]
for dist in range(num_of_districts):
  temp = data2[:,:,dist]
  dst = windowed_dataset(temp, window_size, 1, shuffle_buffer_size,False)
  for x,y in dst:
    X.append(x.numpy())
    Y.append(y.numpy()) 

In [ ]:
X_test,Y_test=[],[]
X_test_sw,Y_test_sw=[],[]
for dist in range(num_of_districts):
  temp = test[:,:,dist]
  dst = windowed_dataset(temp, window_size, 1, shuffle_buffer_size,True)
  x_dist,y_dist=[],[]
  for x,y in dst:
    X_test.append(x.numpy())
    Y_test.append(y.numpy())
    x_dist.append(x.numpy())
    y_dist.append(y.numpy())
  X_test_sw.append(x_dist)
  Y_test_sw.append(y_dist)
X_train=np.squeeze(np.array(X))
Y_train=np.squeeze(np.array(Y))
X_test=np.squeeze(np.array(X_test))
Y_test=np.squeeze(np.array(Y_test))
print(X_test.shape,Y_test.shape)
print(X_train.shape,Y_train.shape)

(95680, 21, 5) (95680, 7, 5)
(127328, 21, 5) (127328, 7, 5)


In [ ]:
np.array(X_test_sw).shape
np.array(Y_test_sw).shape

(736, 130, 7, 5)

In [ ]:
batch_size = 32
shuffle_buffer_size = 1000
traindays=200
num_of_districts=36
window_size=21  #same as n_past
n_past = 21
n_future = 7
n_features = 2
!gdown --id 1VSzHDBC8Xp4PQD-MlWXQAOMUHCsVzRCQ
!gdown --id 1uTXa84C-kX6XZk2QBoadED-fq4HlfeUw
!gdown --id 1Jjeix6c0Oh5g1I0xkRMKCgoapLdVVMHE
!gdown --id 1LkQW3U4nAcl8YG5R_E7wj5kiTpBtN7nc
X_train=np.load('/content/X_train.npy')
Y_train=np.load('/content/Y_train.npy')
X_test=np.load('/content/X_test.npy')
Y_test=np.load('/content/Y_test.npy')

In [ ]:
X_temptrain=X_train[:,:,[3,4]]  #choose active cases and state id
Y_temptrain=np.expand_dims(Y_train[:,:,3],2)
Y_temptest=np.expand_dims(Y_test[:,:,3],2)
X_temptest=X_test[:,:,[3,4]]
print(np.array(X_temptest).shape,np.array(Y_temptest).shape)
print(np.array(X_temptrain).shape,np.array(Y_temptrain).shape)

(95680, 21, 2) (95680, 7, 1)
(127328, 21, 2) (127328, 7, 1)


#LSTM encoder decoder


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(32,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(32, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 = tf.keras.layers.LSTM(32, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(32, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder_l2)
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
model_e2d2.summary()
def mean_absolute_percentage_error2(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
for epochno in range(10):
  print("Epoch number: {}".format(epochno))
  model_e2d2.compile(loss=tf.keras.losses.Huber(),optimizer='adam'
                ,metrics=[tf.keras.losses.MeanAbsolutePercentageError()],run_eagerly=True)
  model_e2d2.fit(X_temptrain,Y_temptrain,batch_size=128, epochs=1)
  model_e2d2.save_weights("weights-{}.h5".format(epochno))
  predictions=model_e2d2.predict(X_temptest)
  print('Performance on test data: ',mean_absolute_percentage_error2(scaler.inverse_transform(predictions.reshape(-1,1)),
                                          scaler.inverse_transform(Y_temptest.reshape(-1,1))))

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 21, 2)]      0                                            
__________________________________________________________________________________________________
lstm_48 (LSTM)                  [(None, 21, 32), (No 4480        input_16[0][0]                   
__________________________________________________________________________________________________
lstm_49 (LSTM)                  [(None, 32), (None,  8320        lstm_48[0][0]                    
__________________________________________________________________________________________________
repeat_vector_15 (RepeatVector) (None, 7, 32)        0           lstm_49[0][0]                    
___________________________________________________________________________________________

#Bidirectional LSTM encoder decoder

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 =  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences = True, return_state=True))
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 =  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM( 32, return_state=True))
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 =  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(decoder_inputs,initial_state = encoder_states1)
decoder_l2 =  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True))(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder_l2)
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
model_e2d2.summary()
def mean_absolute_percentage_error2(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
for epochno in range(30):
  print("Epoch number: {}".format(epochno))
  model_e2d2.compile(loss=tf.keras.losses.Huber(),optimizer='adam'
                ,metrics=[tf.keras.losses.MeanAbsolutePercentageError()],run_eagerly=True)
  model_e2d2.fit(X_temptrain,Y_temptrain,batch_size=128, epochs=1)
  model_e2d2.save_weights("/content/Model1/weights-{}.h5".format(epochno))
  predictions=model_e2d2.predict(X_temptest)
  print('Performance on test data: ',mean_absolute_percentage_error2(scaler.inverse_transform(predictions.reshape(-1,1)),
                                          scaler.inverse_transform(Y_temptest.reshape(-1,1))))

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 21, 2)]      0                                            
__________________________________________________________________________________________________
bidirectional_70 (Bidirectional [(None, 21, 64), (No 8960        input_20[0][0]                   
__________________________________________________________________________________________________
bidirectional_71 (Bidirectional [(None, 64), (None,  24832       bidirectional_70[0][0]           
__________________________________________________________________________________________________
repeat_vector_19 (RepeatVector) (None, 7, 64)        0           bidirectional_71[0][0]           
___________________________________________________________________________________________

In [ ]:
!gdown --id 1-XU-T_fWIEUITd1bBFTz2x7QFUlJStgq

Downloading...
From: https://drive.google.com/uc?id=1-XU-T_fWIEUITd1bBFTz2x7QFUlJStgq
To: /content/weights-5_28.4mape21_7.h5
100% 392k/392k [00:00<00:00, 56.3MB/s]


In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 =  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences = True, return_state=True))
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 =  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM( 32, return_state=True))
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 =  tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tfa.rnn.PeepholeLSTMCell(32),return_sequences = True))(decoder_inputs,initial_state = encoder_states1)
decoder_l2 =  tf.keras.layers.Bidirectional(tf.keras.layers.RNN( tfa.rnn.PeepholeLSTMCell(32),return_sequences=True))(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder_l2)
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
model_e2d2.summary()
def mean_absolute_percentage_error2(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
for epochno in range(30):
  print("Epoch number: {}".format(epochno))
  model_e2d2.compile(loss=tf.keras.losses.Huber(),optimizer='adam'
                ,metrics=[tf.keras.losses.MeanAbsolutePercentageError()],run_eagerly=True)
  model_e2d2.fit(X_temptrain,Y_temptrain,batch_size=128, epochs=1)
  # model_e2d2.save_weights("weights-{}.h5".format(epochno))
  predictions=model_e2d2.predict(X_temptest)
  print('Performance on test data: ',mean_absolute_percentage_error2(scaler.inverse_transform(predictions.reshape(-1,1)),
                                          scaler.inverse_transform(Y_temptest.reshape(-1,1))))

In [ ]:
X_test_sw=np.array(X_test_sw)
Y_test_sw=np.array(Y_test_sw)
print(Y_test_sw.shape,
X_test_sw.shape)

(736, 130, 7) (736, 130, 21, 2)


In [ ]:
X_test_sw=X_test_sw[:,:,:,[3,4]]
Y_test_sw=Y_test_sw[:,:,:,3]

In [ ]:
x=X_test_sw[1:10]
print(x.shape,
x.shape)
value=[1,9]
x=X_test_sw[value[0]:value[1]].reshape(X_test_sw[value[0]:value[1]].shape[0]*X_test_sw[value[0]:value[1]].shape[1],21,5)
print(x.shape)

(9, 130, 21, 5) (9, 130, 21, 5)
(1040, 21, 5)


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
l=[]
# predictions_rs =model_e2d2.predict(X_temptest)
for key, value in dicti.items():
  x=X_test_sw[value[0]:value[1]].reshape(X_test_sw[value[0]:value[1]].shape[0]*X_test_sw[value[0]:value[1]].shape[1],21,2)
  y=Y_test_sw[value[0]:value[1]]
  predictions=model_e2d2.predict(x)
  mapes=mean_absolute_percentage_error(scaler.inverse_transform(predictions.reshape(-1,1)),
                                          scaler.inverse_transform(y.reshape(-1,1)))
  print('Mape for {} is {}'.format(key,mapes))
np.sort(np.array(l).astype(int))

Mape for Andhra_Pradesh is 18.7102917497113
Mape for Arunachal_Pradesh is 70.17960074474192
Mape for Assam is 40.260853960802464
Mape for Bihar is 23.726293331788284
Mape for Chattisgarh is 38.458527446461375
Mape for Goa is 54.31766942908578
Mape for Gujarat is 28.6005770114837
Mape for Haryana is 19.841503458030093
Mape for Himanchal_Pradesh is 22.735887516991376
Mape for Jharkhand is 33.559017801001616
Mape for Karnataka is 22.5363451582208
Mape for Kerala is 8.517367238751463
Mape for Madhya_Pradesh is 27.95267779030242
Mape for Maharashtra is 19.613015521819854
Mape for Manipur is 36.07555191169534
Mape for Meghalaya is 51.88851515179407
Mape for Mizoram is 58.12935327611595
Mape for Nagaland is 61.9596380022987
Mape for Odisha is 23.830565065966397
Mape for Punjab is 20.049227664130516
Mape for Rajasthan is 23.88149263551798
Mape for Sikkim is 96.34302754865942
Mape for Tamil_Nadu is 22.01417373235579
Mape for Telangana is 43.701800575248534
Mape for Tripura is 24.843338287805512

array([], dtype=int64)

# Bi-directional Peephole encoder-decoder

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 =  tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tfa.rnn.PeepholeLSTMCell(32),return_sequences = True,return_state=True))
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 =  tf.keras.layers.Bidirectional(tf.keras.layers.RNN( tfa.rnn.PeepholeLSTMCell(32),return_state=True))
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 =  tf.keras.layers.Bidirectional(tf.keras.layers.RNN(tfa.rnn.PeepholeLSTMCell(32),return_sequences = True))(decoder_inputs,initial_state = encoder_states1)
decoder_l2 =  tf.keras.layers.Bidirectional(tf.keras.layers.RNN( tfa.rnn.PeepholeLSTMCell(32),return_sequences=True))(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1))(decoder_l2)
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
model_e2d2.summary()
!gdown --id 1-XU-T_fWIEUITd1bBFTz2x7QFUlJStgq
model_e2d2.load_weights('/content/weights-5_28.4mape21_7.h5')

# def mean_absolute_percentage_error2(y_true, y_pred): 
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# for epoch in range(100):
#   epochno=epoch+27
#   print("Epoch number: {}".format(epochno))
#   model_e2d2.compile(loss=tf.keras.losses.Huber(),optimizer=tf.optimizers.Adam(0.00001),
#   metrics=[tf.keras.losses.MeanAbsolutePercentageError()], 
#                      run_eagerly=True)
#   model_e2d2.fit(X_temptrain,Y_temptrain,batch_size=128, epochs=1)
#   model_e2d2.save_weights("/content/drive/MyDrive/DST/weights-{}.h5".format(epochno))
#   predictions=model_e2d2.predict(X_temptest)
#   print('Performance on test data: ',mean_absolute_percentage_error2(scaler.inverse_transform(predictions.reshape(-1,1)),
#                                           scaler.inverse_transform(Y_temptest.reshape(-1,1))))

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 21, 2)]      0                                            
__________________________________________________________________________________________________
bidirectional_62 (Bidirectional [(None, 21, 64), (No 9152        input_18[0][0]                   
__________________________________________________________________________________________________
bidirectional_63 (Bidirectional [(None, 64), (None,  25024       bidirectional_62[0][0]           
__________________________________________________________________________________________________
repeat_vector_17 (RepeatVector) (None, 7, 64)        0           bidirectional_63[0][0]           
___________________________________________________________________________________________

In [ ]:
predictions=model_e2d2.predict(X_temptest)
print('Performance on test data: ',mean_absolute_percentage_error2(scaler.inverse_transform(predictions.reshape(-1,1)),
                                          scaler.inverse_transform(Y_temptest.reshape(-1,1))))

Performance on test data:  28.48894955502333


In [ ]:
dicti ={
 'Andhra_Pradesh':[0,13],
 'Arunachal_Pradesh':[13,38],
 'Assam':[38,71],
 'Bihar':[71,109],
 'Chattisgarh':[109,137],
 'Goa':[137,139],
 'Gujarat':[139,172],
 'Haryana':[172,194],
 'Himanchal_Pradesh':[194,206],
 'Jharkhand':[206,230],
 'Karnataka':[230,260],
 'Kerala':[260,274],
 'Madhya_Pradesh':[274,329],
 'Maharashtra':[329,365],
 'Manipur':[365,381],
 'Meghalaya':[381,392],
 'Mizoram':[392,400],
 'Nagaland':[400,412],
 'Odisha':[412,442],
 'Punjab':[442,464],
 'Rajasthan':[464,497],
 'Sikkim':[497,501],
 'Tamil_Nadu':[501,539],
 'Telangana':[539,572],
 'Tripura':[572,580],
 'Uttar_Pradesh':[580,655],
 'Uttarakhand':[655,668],
 'West_Bengal':[668,691],
 'Andaman':[691,694],
 'Chandigarh':[694,695],
 'Daman':[695,698],
 'Jammu':[698,718],
 'Ladakh':[718,720],
 'Lakshadweep':[720,721],
 'Delhi':[721,732],
 'Puducherry':[732,736]}
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
l=[]
predictions_rs =model_e2d2.predict(X_temptest)
for key, value in dicti.items():
  mapes=mean_absolute_percentage_error(scaler.inverse_transform(predictions[value[0]:value[1],:].reshape(-1,1)),
                                          scaler.inverse_transform(Y_temptest[value[0]:value[1],:].reshape(-1,1)))
  print('Mape for {} is {}'.format(key,mapes))
np.sort(np.array(l).astype(int))

In [ ]:
!git clone https://github.com/titu1994/Keras-Multiplicative-LSTM

Cloning into 'Keras-Multiplicative-LSTM'...
remote: Enumerating objects: 27, done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 27
Unpacking objects: 100% (27/27), done.


In [ ]:
!pip install q keras==2.3.1
!pip install tensorflow==1.14
import keras

In [ ]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/Keras-Multiplicative-LSTM')

In [ ]:
from multiplicative_lstm import MultiplicativeLSTM

In [ ]:
X_train=np.load('/content/X_train.npy')
Y_train=np.load('/content/Y_train.npy')
X_test=np.load('/content/X_test.npy')
Y_test=np.load('/content/Y_train.npy')

In [ ]:
encoder_inputs = keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = keras.layers.Bidirectional(MultiplicativeLSTM( 32,return_sequences = True, return_state=True))
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = keras.layers.Bidirectional(MultiplicativeLSTM(32, return_state=True))
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
decoder_inputs = keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
decoder_l1 =  keras.layers.Bidirectional(MultiplicativeLSTM(32, return_sequences=True))( decoder_inputs,initial_state = encoder_states1)
decoder_l2 = keras.layers.Bidirectional(MultiplicativeLSTM(32, return_sequences=True ))(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = keras.layers.TimeDistributed(keras.layers.Dense(1))(decoder_l2)
model_e2d22 = keras.models.Model(encoder_inputs,decoder_outputs2)
model_e2d22.summary()

In [ ]:
encoder_inputs = keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = keras.layers.Bidirectional(MultiplicativeLSTM( 32,return_sequences = True, return_state=True))
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = keras.layers.Bidirectional(MultiplicativeLSTM(32, return_state=True))
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
decoder_inputs = keras.layers.RepeatVector(n_future)(encoder_outputs2[0])

decoder_l1 = MultiplicativeLSTM(32, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = MultiplicativeLSTM(32, return_sequences=True )(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = keras.layers.TimeDistributed(keras.layers.Dense(1))(decoder_l2)
model_e2d22 = keras.models.Model(encoder_inputs,decoder_outputs2)
model_e2d22.summary()
# !gdown --id 1-XU-T_fWIEUITd1bBFTz2x7QFUlJStgq
# model_e2d2.load_weights('/content/weights-5_28.4mape21_7.h5')
def mean_absolute_percentage_error2(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
for epoch in range(100):
  epochno=epoch
  print("Epoch number: {}".format(epochno))
  model_e2d22.compile(loss=keras.losses.Huber(),optimizer=keras.optimizers.Adam(0.0001),
  metrics=[keras.losses.MeanAbsolutePercentageError()])
  model_e2d22.fit(X_temptrain,Y_temptrain,batch_size=128, epochs=1)
  # model_e2d22.save_weights("/content/drive/MyDrive/DST/weights-{}.h5".format(epochno))
  predictions=model_e2d22.predict(X_temptest)
  print('Performance on test data: ',mean_absolute_percentage_error2(scaler.inverse_transform(predictions.reshape(-1,1)),
                                          scaler.inverse_transform(Y_temptest.reshape(-1,1))))

ValueError: ignored